In [2]:
import os
import json
import pandas as pd
import traceback 


In [3]:
from langchain.chat_models import ChatOpenAI


In [6]:
key1= os.getenv("OPENAI_API_KEY")
#print(key1)

In [7]:
from dotenv import load_dotenv
load_dotenv()  # take env variable from .env

key=os.getenv("OPENAI_API_KEY")
#print(key)

In [55]:
llm =ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)
#llm

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [13]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template= TEMPLATE
)

In [14]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

c:\Users\kevin\OneDrive\Documents\Learning\Youtube\GenAI\GenAI_project1\genaipro_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [17]:
review_chain =LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose= True)

In [18]:
generate_evaluate_chain= SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                         output_variables=["quiz","review"],verbose=True)

In [19]:
filepath= r"C:\Users\kevin\OneDrive\Documents\Learning\Youtube\GenAI\GenAI_project1\data.txt"
with open(filepath,'r') as file:
    TEXT= file.read()

print(TEXT)


# //or
# current_dir = os.getcwd()
# parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
# file_path = os.path.join(parent_dir, 'data.txt')

# with open(file_path, 'r') as file:
#     content = file.read()

# print(content)

Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gray imag

In [20]:
json.dumps(RESPONSE_JSON)  # serialize python dic into a json formatted string


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [26]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However

In [27]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1774
Prompt Tokens:1274
Completion Tokens:500
Total Cost:0.002911


In [28]:
response

{'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner

In [30]:
quiz_str=response.get('quiz')
quiz_dict =json.loads(quiz_str)  # creates dict

{'1': {'mcq': 'What is data science?',
  'options': {'a': 'A field that uses statistics and scientific computing to extract knowledge from data',
   'b': 'A branch of computer science focused on data visualization',
   'c': 'A discipline that studies ancient data systems',
   'd': 'A type of art form using data as the medium'},
  'correct': 'a'},
 '2': {'mcq': 'What does a data scientist do?',
  'options': {'a': 'Create programming code and combine it with statistical knowledge to gain insights from data',
   'b': 'Design websites and mobile applications',
   'c': 'Conduct experiments in a laboratory setting',
   'd': 'Write novels based on data analysis'},
  'correct': 'a'},
 '3': {'mcq': 'What skills are required in data science?',
  'options': {'a': 'Computer science, statistics, information science, and mathematics',
   'b': 'Graphic design, complex systems, and communication',
   'c': 'Only computer science and statistics',
   'd': 'None of the above'},
  'correct': 'a'},
 '4': {'

In [51]:
# "==".join([f"{i}:{i*i}" for i in range(10)])
quiz_table_data  =[]
for key, value in quiz_dict.items():
    mcq=value['mcq']
    option = " | ".join(
        [f"{key1}:{value1}" for key1,value1 in value['options'].items()]
    )
    correct =value['correct']
    quiz_table_data.append({"MCQ":mcq,"Choices": option,"Correct":correct})

quiz_table_data





[{'MCQ': 'What is data science?',
  'Choices': 'a:A field that uses statistics and scientific computing to extract knowledge from data | b:A branch of computer science focused on data visualization | c:A discipline that studies ancient data systems | d:A type of art form using data as the medium',
  'Correct': 'a'},
 {'MCQ': 'What does a data scientist do?',
  'Choices': 'a:Create programming code and combine it with statistical knowledge to gain insights from data | b:Design websites and mobile applications | c:Conduct experiments in a laboratory setting | d:Write novels based on data analysis',
  'Correct': 'a'},
 {'MCQ': 'What skills are required in data science?',
  'Choices': 'a:Computer science, statistics, information science, and mathematics | b:Graphic design, complex systems, and communication | c:Only computer science and statistics | d:None of the above',
  'Correct': 'a'},
 {'MCQ': 'How is data science different from computer science and information science?',
  'Choices':

In [53]:
df=pd.DataFrame(quiz_table_data)
df

,MCQ,Choices,Correct
0,What is data science?,a:A field that uses statistics and scientific ...,a
1,What does a data scientist do?,a:Create programming code and combine it with ...,a
2,What skills are required in data science?,"a:Computer science, statistics, information sc...",a
3,How is data science different from computer sc...,a:It focuses on extracting knowledge from data...,a
4,What did Turing Award winner Jim Gray describe...,a:A branch of computer science | b:A type of a...,c


In [54]:
df.to_csv("Data_Science_Quiz.csv",index=False)

In [56]:
os.getcwd()

'c:\\Users\\kevin\\OneDrive\\Documents\\Learning\\Youtube\\GenAI\\GenAI_project1\\experiment'

In [70]:
os.path.join(os.getcwd(),"log")
os.makedirs(os.path.join(os.getcwd(),"log"),exist_ok=True)